In [2]:
import requests
import time
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy
from dotenv import load_dotenv
import os

load_dotenv()

True

### Verbindung zur Datenbank aufbauen und Daten aus der Tabelle laden

In [3]:
host = os.getenv('HOST')
port = os.getenv('PORT')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
database= os.getenv('DATABASE')

In [4]:
# Verbindung zur Datenbank herstellen
connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
print("Verbindung zur Datenbank erfolgreich hergestellt.")

# Einen Cursor erstellen, um SQL-Abfragen auszuführen
cursor = connection.cursor()

# SQL SELECT-Abfrage
select_query = "SELECT * FROM value;"

# Abfrage ausführen
cursor.execute(select_query)

# Ergebnis abrufen
rows = cursor.fetchall()

# Ergebnis in ein Pandas DataFrame umwandeln
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Cursor und Verbindung schließen
cursor.close()
connection.close()


Verbindung zur Datenbank erfolgreich hergestellt.


### Random Forest Modell

In [5]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error


# 1. Convert the time column to datetime objects
df['time'] = pd.to_datetime(df['time'])

# 2. Feature Engineering with Lags
lags = [1, 2, 3]  # Lags for 10, 20, and 30 minutes
for lag in lags:
    df[f'Temperatur_lag_{lag*10}'] = df.groupby('senseid_fk')['temperature'].shift(lag)

# 3. Create target variables for each prediction horizon
for minutes in [10, 20, 30]:
    shift = minutes // 10  # Calculate the shift based on 10-minute intervals
    df[f'Temperatur_in_{minutes}_Minuten'] = df.groupby('senseid_fk')['temperature'].shift(-shift)

# 4. Remove rows with missing values (introduced by lagging and shifting)
df.dropna(inplace=True)

In [8]:
# 5. Define target variables and create a dictionary for models
target_variables = ['Temperatur_in_10_Minuten', 'Temperatur_in_20_Minuten', 'Temperatur_in_30_Minuten']
models = {}

# 6. Train and evaluate models for each target variable
for target in target_variables:
    # 6.1. Select features and target variable
    X = df.drop(target_variables + ["time", "senseid_fk"], axis=1)
    y = df[target]

    # 6.2. Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 6.3. Create and train the Random Forest model
    model = RandomForestRegressor(n_estimators=400,max_depth=40, random_state=42)
    model.fit(X_train, y_train)

    # 6.4. Make predictions and evaluate
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"\n{target}:")
    print(f"  Mean Squared Error: {mse}")
    print(f"  Mean Absolute Error: {mae}")

    # 6.5. Store the trained model
    models[target] = model


Temperatur_in_10_Minuten:
  Mean Squared Error: 0.6959775182910382
  Mean Absolute Error: 0.29405328315365437

Temperatur_in_20_Minuten:
  Mean Squared Error: 1.1561430452437964
  Mean Absolute Error: 0.42871305138224786

Temperatur_in_30_Minuten:
  Mean Squared Error: 1.5624276179679655
  Mean Absolute Error: 0.5264186126191158


### Mit GridSearch und CV

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.metrics import mean_squared_error, mean_absolute_error

# # 5. Define target variables and create a dictionary for models
# target_variables = ['Temperatur_in_10_Minuten', 'Temperatur_in_20_Minuten', 'Temperatur_in_30_Minuten']
# models = {}

# # Hyperparameter grid for Random Forest
# param_grid = {
#     'n_estimators': [300, 400],
#     'max_depth': [30, 40, 50, 60, 70],
# }

# # 6. Train and evaluate models for each target variable
# for target in target_variables:
#     # 6.1. Select features and target variable
#     X = df.drop(target_variables + ["time", "senseid_fk"], axis=1)
#     y = df[target]

#     # 6.2. Split data into training and testing sets
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     # 6.3. Perform GridSearchCV to find the best hyperparameters
#     rf = RandomForestRegressor(random_state=42)
#     grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
#     grid_search.fit(X_train, y_train)

#     # 6.4. Get the best model from grid search
#     best_model = grid_search.best_estimator_

#     # 6.5. Make predictions and evaluate
#     y_pred = best_model.predict(X_test)
#     mse = mean_squared_error(y_test, y_pred)
#     mae = mean_absolute_error(y_test, y_pred)
#     print(f"\n{target}:")
#     print(f"  Best Parameters: {grid_search.best_params_}")
#     print(f"  Mean Squared Error: {mse}")
#     print(f"  Mean Absolute Error: {mae}")

#     # 6.6. Store the trained model
#     models[target] = best_model



Temperatur_in_10_Minuten:
  Best Parameters: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
  Mean Squared Error: 0.7231009365555813
  Mean Absolute Error: 0.29696808916272927

Temperatur_in_20_Minuten:
  Best Parameters: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
  Mean Squared Error: 1.1219574791632885
  Mean Absolute Error: 0.4293155597508828

Temperatur_in_30_Minuten:
  Best Parameters: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
  Mean Squared Error: 1.3734161227737929
  Mean Absolute Error: 0.5241237983584991
